In [18]:
import os
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient
import pandas as pd

from dotenv import load_dotenv

import warnings
warnings.filterwarnings('ignore')

In [15]:
load_dotenv()
endpoint = os.getenv('AZURE_LANGUAGE_ENDPOINT')
key = os.getenv('AZURE_LANGUAGE_KEY')

textAnalyticsClient = TextAnalyticsClient(endpoint=endpoint, credential=AzureKeyCredential(key=key))

In [16]:
data = pd.read_csv("SubsetBERT.csv")
subset = data[0:5]

reviews = subset['content'].tolist()
response = textAnalyticsClient.analyze_sentiment(documents=reviews)

predictedScores = []
confidenceScores = []

for doc in response:
    predictedScores.append(doc.sentiment) 
    confidenceScores.append(
        max(doc.confidence_scores.__dict__.values()) * 100)

subset['AzurePredictedScore'] = predictedScores
subset['AzureConfidence'] = confidenceScores

/var/folders/xs/m589rd993f3_8k0zh1qk8vkw0000gn/T/ipykernel_66803/1425677297.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['AzurePredictedScore'] = predictedScores
/var/folders/xs/m589rd993f3_8k0zh1qk8vkw0000gn/T/ipykernel_66803/1425677297.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['AzureConfidence'] = confidenceScores


In [19]:
subset.head()

,id,date,content,score,userName,app,platform,predictedScore,confidence,AzurePredictedScore,AzureConfidence
0,687616,2020-11-03 20:54:57,['ok'],4,Donovan Scully,Uber Eats,Google Play,3,49.511460,neutral,84.0
1,545363,2021-11-13 23:58:22,"['absolutely', 'best', 'app', 'keep', 'sit', '...",5,Lee Turner,Uber Eats,Google Play,5,86.219880,neutral,56.0
2,1036517,2019-01-29 08:58:12,"['convient', 'useful']",5,A Google user,Uber Eats,Google Play,4,53.396255,neutral,54.0
3,1220389,2017-01-27 03:43:12,"['great', 'however', 'would', 'well', 'place',...",4,A Google user,Uber Eats,Google Play,4,53.166737,neutral,49.0
4,1243942,2020-05-05 12:07:15,"['ridiculous', 'pricing', 'policy', 'czech', '...",2,Tony P,Wolt,Google Play,1,61.972828,negative,82.0
